# Loan Summary Table Routine

#### This script runs everyday and inserts summary of loans_loan table into loans_loansummary table.

In [1]:
import os
import pandas as pd

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

loans_df = pd.DataFrame(list(Loan.objects.filter(active=True).values()))
loan_types = loans_df.loan_type_id.unique()
banks = loans_df.bank_id.unique()

for loan_type in loan_types:
    print("working on: "+loan_type)
    for bank in banks:
        # try:
        filtered_df = loans_df[(loans_df.bank_id==bank)&(loans_df.loan_type_id==loan_type)&(loans_df.is_base==False)]
        if not filtered_df.shape[0]==0:
            min_max_principal = filtered_df.principal_id.aggregate(['min','max'])
            min_max_tenure = filtered_df.tenure_id.aggregate(['min','max'])
            min_max_interest = filtered_df.interest.aggregate(['min','max'])
            # Introduce a LoanSummary object and save to db
            ls = LoanSummary()
            ls.bank = Bank.objects.get(name=bank)
            ls.loan_type = LoanType.objects.get(loan_type=loan_type)
            ls.min_principal = Principal.objects.get(principal=min_max_principal['min'])
            ls.max_principal = Principal.objects.get(principal=min_max_principal['max'])
            ls.min_tenure = Tenure.objects.get(tenure=min_max_tenure['min'])
            ls.max_tenure = Tenure.objects.get(tenure=min_max_tenure['max'])
            ls.min_interest = min_max_interest['min']
            ls.max_interest = min_max_interest['max']
            # we need to add available tenures:
            ls.save()
            tenures = filtered_df.tenure_id.unique()
            for this_tenure in tenures:
                ls.tenures.add(Tenure.objects.get(tenure=this_tenure))
            ls.save()
            print("saved for bank: "+bank)
        else:
            print("no loan found for bank: "+bank)
        #except:
        #    print("error in bank : "+bank)

working on: CAR
saved for bank: Garanti Bankası
no loan found for bank: ING Bank
no loan found for bank: İş Bankası
saved for bank: Finansbank
saved for bank: Yapı ve Kredi Bankası
working on: GPL
saved for bank: Garanti Bankası
saved for bank: ING Bank
saved for bank: İş Bankası
saved for bank: Finansbank
no loan found for bank: Yapı ve Kredi Bankası
working on: CA2
no loan found for bank: Garanti Bankası
saved for bank: ING Bank
no loan found for bank: İş Bankası
no loan found for bank: Finansbank
no loan found for bank: Yapı ve Kredi Bankası
working on: MOR
no loan found for bank: Garanti Bankası
no loan found for bank: ING Bank
saved for bank: İş Bankası
no loan found for bank: Finansbank
no loan found for bank: Yapı ve Kredi Bankası


### Below is Junk for now

In [ ]:
banks_df = pd.DataFrame(list(Bank.objects.filter(active=True).values()))

In [ ]:
loan_types = list([e['loan_type'] for e in LoanType.objects.values('loan_type')])
bank = 'ING Bank'
loan_type = loan_types[0]
filtered_df = loans_df[(loans_df.bank_id==bank)&(loans_df.loan_type_id==loan_type)]

In [ ]:
principals = filtered_df.principal_id.unique()
principal = principals[0]

In [ ]:
filtered_df = filtered_df[filtered_df.principal_id<=principal]
tenures = sorted(filtered_df.tenure_id.unique())

In [ ]:
for i in range(1,len(tenures)):
    ls = LoanSummary()
    ls.bank_name = bank
    ls.bank_logo_path = banks_df[banks_df.name==ls.bank_name].logo_path
    ls.loan_name = filtered_df[filtered_df.tenure_id==tenures[i]].name
    ls.min_principal = filtered_df[filtered_df.tenure_id==tenures[i-1]].principal_id
    ls.max_principal = filtered_df[filtered_df.tenure_id==tenures[i]].principal_id
    ls.min_tenure = tenures[i-1]+1 # FRAGILE CODE
    ls.max_tenure = tenures[i]
    ls.interest = filtered_df[filtered_df.tenure_id==tenures[i]].interest
    ls.save()

In [ ]:
import os
import pandas as pd

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [ ]:
loan_summary = LoanSummary.objects.all().order_by('-min_interest')

In [ ]:
loan_summary[0].min_principal.principal

In [ ]:
loan_summary[0].min_principal.principal|as_currency_plain 

In [ ]:
from babel.numbers import format_currency

In [ ]:
"aaaa".replace(',','.')

In [ ]:
pd.DataFrame(list(loan_summary.values()))

In [ ]:
loan_summary = LoanSummary.objects.all().order_by('-min_interest')

In [ ]:
def sort_tenures_list(tenures):
    

In [ ]:
dict(zip([e.id for e in loan_summary],[list(e.tenures.all().order_by('tenure')) for e in loan_summary]))